In [1]:
using Oscar

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.10.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


## Exercise 1: Normalization and how it can be used

This exercise is merely to make you aquinted with normalization and how it can be used to get more insight 
into singularities.

As an example consider the Whitney umbrella given by the zero locus of $f = z^2 - x*y^2$.

 * Find the parametrization via `normalization` and `elimpart` as shown in the lecture
 * Explicitly write down the parametrization.

When finished, repeat the process with $h = x^3 - x^2*z + x*y^2 - x*z^2 - y^2*z + z^3$. What do you get?
Compare this with the output of the primary decomposition: What is the geometry reflected by the normalization?

In [57]:
R, (x,y,z) = QQ["x", "y", "z"]
f = z^2 - x*y^2                 # The equation of the Whitney umbrella
I = ideal(R, f)
A, _ = quo(R, I)                # The coordinate ring of the Whitney umbrella as an 
                                # affine variety (not as a germ!)
output = normalization(A)       # `output` is a complicated list; see the OSCAR documentation
                                # for details.
B = output[1][1]                # This is the coordinate ring of the normalization
elimpart(modulus(B))            # `elimpart` will try to eliminate variables and simplify 
                                # the presentation of `B`.

eliminating superfluous variables from the ideal ideal(T(1)*y - z, -T(1)*z + x*y, T(1)^2 - x, x*y^2 - z^2)
Keep the variables ["T(1)", "y"];
Substitute as follows:
T(1) -> T(1)
x -> T(1)^2
y -> y
z -> T(1)*y
After substitution, the ideal is generated by fmpq_mpoly[]


(Multivariate Polynomial Ring in T(1), y over Rational Field, ideal(0), Map with following data
Domain:
Multivariate Polynomial Ring in T(1), x, y, z over Rational Field
Codomain:
Multivariate Polynomial Ring in T(1), y over Rational Field)

## Exercise 2: Finite determinacy

Recall the bounds for finite determinacy for isolated hypersurface singularities for $\mathcal R$ and $\mathcal K$ equivalence from the lecture. Write a function `is_R_k_determined(f::MPolyElem, k::Int)` which checks whether 
a polynomial $f$ is $k$-determined for $\mathcal R$-equivalence; repeat the same for $\mathcal K$-equivalence. 

You can test you functions on the singularities given by $f(x,y,z) = x^3 + y^3 + z^3 + x*y*z$ and $g(x,y) = x^5 + y^{11}$.

In [58]:
# Set up the two test polynomials
f = x^3 + y^3 + z^3 + x*y*z
g = x^5 + y^11 +z^2

function is_R_k_determined(f::MPolyElem, k::Int)
    R = parent(f)
    m = ideal(R,gens(R)) # the maximal ideal at the origin
    m_k_plus_one = m     # a variable that will in the end contain the k+1-st 
                         # power of the maximal ideal 
    
    # the following loop is a manual implementation for taking powers of ideals
    for i in 1:k
        m_k_plus_one = ideal(R, [h*g for h in gens(m) for g in gens(m_k_plus_one)])
    end
    
    J = ideal(R,minors(jacobi_matrix(f),1))
    
    # the next command is a shortcut for taking powers and products of ideals
    m_sq_J = m^2*J
    
    # We need to check the containment of ideals according to the formula from 
    # the lecture
    for g in gens(m_k_plus_one)
        if !(g in m_sq_J)
            return false
        end
    end
            
    return true
end

is_R_k_determined (generic function with 1 method)

In [60]:
# test the determinacy with the above polynomials
is_R_k_determined(f, 3)

true

In [59]:
# an alternative implementation: 
function is_R_k_determined(f::MPolyElem, k::Int)
    R = parent(f)
    m = ideal(R, gens(R))
    J = ideal(R,minors(jacobi_matrix(f),1))
    return issubset(m^(k+1), J*m^2)
end

is_R_k_determined (generic function with 1 method)

## Exercise 3: Versal deformations for IHS

You saw in the lectures how to one can write a function to automatically generate 
a semi-universal unfolding $F$ for $\mathcal R$-equivalence of an isolated hypersurface singularity $f$. 

 * Do the same for a semi-universal deformation of $X = f^{-1}(\{0\})$, i.e. up to $\mathcal K$-equivalence. 

 * When done, test your function on the $D_4$-singularity given by $x*(x^2 - y^2)+ z^2$. What is the discriminant?

 * Find the semi-universal deformation of the singularity $h = x^5 + y^5 + x^3*y^3$. How does that compare to the semi-universal unfolding (i.e. up to $\mathcal R$ equivalence)? 
 
**WARNING:** It is a hopeless undertaking to compute the discriminants in the last exercise!

In [62]:
f = x*(x^2 - y^2)+ z^2

# A function for the ideal of the Tjurina algebra
function T1_ideal(f::MPolyElem)
    R = parent(f)
    Jext = ideal(R, minors(jacobi_matrix(f), 1)) + ideal(R, f)
    return Jext
end

# This function returns a list of monomials which 
# reduce to a basis of the vector space (R/I)_m for a 
# zero dimensional ideal I in a multivariate polynomial 
# ring R over a field. 
function monomial_basis(I::MPolyIdeal)
    R = base_ring(I)
    o = negdegrevlex(gens(R)) # make sure to use a local ordering! 
    lead_I = leading_ideal(I, ordering=o)
    result = []
    d = 0
    new_mons = [m for m in Oscar.all_monomials(R, d) if !(m in lead_I)]
    while length(new_mons) > 0
        # println("new monomials found in degree $d: $new_mons") # uncomment for verbose output
        result = vcat(result, new_mons)
        d = d + 1
        new_mons = [m for m in Oscar.all_monomials(R, d) if !(m in lead_I)]
    end
    return result
end
    
Jext = T1_ideal(f)
monbase = monomial_basis(Jext)

4-element Vector{Any}:
 1
 x
 y
 y^2

In [63]:
function semi_universal_deformation(f::MPolyElem)
    R = parent(f)
    monbase = monomial_basis(T1_ideal(f))
    # prepare a new ring containing also the deformation parameters as variables
    S, v = PolynomialRing(coefficient_ring(R), vcat(symbols(R), Symbol.(["u$i" for i in 1:length(monbase)]))) 

    # the following map is needed to relate R to the new ring
    R_to_S = hom(R, S, v[1:ngens(R)])

    F = R_to_S(f)

    for i in 1:length(monbase)
        F = F + v[ngens(R) + i]*R_to_S(monbase[i]);
    end
    return F
end


semi_universal_deformation (generic function with 1 method)

In [55]:
semi_universal_deformation(f)

new monomials found in degree 0: fmpq_mpoly[1]
new monomials found in degree 1: fmpq_mpoly[x, y]
new monomials found in degree 2: fmpq_mpoly[y^2]


x^3 - x*y^2 + x*u2 + y^2*u4 + y*u3 + z^2 + u1

## Exercise 4: Versal deformations of ICIS

In the lecture's worksheets, there was already a method to compute the $T^1$ of an ICIS. 

 * Extend this functionality to write down a semi-universal deformation.
 * Test your method with the 'fat point' singularity $F_4$ given by the regular sequence $x*y, x^2 + y^3$.

## Exercise 5 (*): GL-equivalence and unfoldings of matrices

Recall the notions of GL-equivalence for $m\times n$-matrices from the lectures by Prof. Maria Ruas: 

$A \sim A' \Leftrightarrow \exists P \in \mathrm{GL}(m), Q \in \mathrm{GL}(n), \Phi \colon (\mathbb C^p,0) \to (\mathbb C^p,0) : A \circ \Phi = P \cdot A' \cdot Q^{-1}$.

 * Write a function which computes the extended tangent space of a given matrix $A$ for GL-equivalence. 
 * Write a function that returns the semi-universal unfolding for GL-equivalence for a given matrix.
 * Can you compute the discriminants in some examples?

## Exercise 6 (*): Bifurcation

Recall the question by Juan Nuno-Ballesteros from the last lecture. 

Let $F: (\mathbb C^n,0) \times (\mathbb C^k,0) \to (\mathbb C,0)$ be the unfolding of a map germ $f \colon (\mathbb C^n,0) \to (\mathbb C,0)$. We write $f_t$ for the function $F(-;t)$.  The 
*bifurcation set* $B \subset \mathbb C^k$ is defined as 
the set of parameters $t \in \mathbb C^k$ for which $f_t$ either has a non-Morse critical point or at least two critical points with the same image.

 * Write a program to compute the equations for the bifurcation set.

(*) hard exercise!